<u>Citation</u><br>
 S. Moro, R. Laureano and P. Cortez. <i>Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. </i>
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, 
  Portugal, October, 2011. EUROSIS.
<hr>
The data is related with direct marketing campaigns of a Portuguese banking institution. 
   The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, 
   in order to access if the product (bank term deposit) would be (or not) subscribed. 

   There are two datasets: 
   <ul>
    <li><b>bank-full.csv</b> with all examples, ordered by date (from May 2008 to November 2010).</li>
    <li><b>bank.csv</b> with 10% of the examples (4521), <i>randomly selected</i> from bank-full.csv.</li>
   </ul>
   <br>
   The smallest dataset is provided to test more computationally demanding machine learning algorithms (e.g. SVM).

In [1]:
# Print all outputs in a block - not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

In [3]:
Calls = pd.read_csv("bank-full.csv", sep=";")
type(Calls)

print("Read a dataset with ", 
      Calls.shape[0], "rows and ",
      Calls.shape[1], "columns...")

Calls.head(5)

# Get header row - uncomment if you want to
# list(Calls.columns)

# Examine the column format
Calls.dtypes

pandas.core.frame.DataFrame

Read a dataset with  45211 rows and  17 columns...


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [4]:
# Convert variables that are incorrectly types
cat_cols = ["job", "marital", "education", "default",
            "housing", "loan", "month", "poutcome", "y"]
Calls[cat_cols] = Calls[cat_cols].astype("category")

In [5]:
Calls.dtypes

age             int64
job          category
marital      category
education    category
default      category
balance         int64
housing      category
loan         category
contact        object
day             int64
month        category
duration        int64
campaign        int64
pdays           int64
previous        int64
poutcome     category
y            category
dtype: object

In [6]:
# Describe the categorical variables
Calls[cat_cols].describe()

,job,marital,education,default,housing,loan,month,poutcome,y
count,45211,45211,45211,45211,45211,45211,45211,45211,45211
unique,12,3,4,2,2,2,12,4,2
top,blue-collar,married,secondary,no,yes,no,may,unknown,no
freq,9732,27214,23202,44396,25130,37967,13766,36959,39922


In [7]:
Calls["job"].unique()

[management, technician, entrepreneur, blue-collar, unknown, ..., services, self-employed, unemployed, housemaid, student]
Length: 12
Categories (12, object): [management, technician, entrepreneur, blue-collar, ..., self-employed, unemployed, housemaid, student]

In [8]:
# Get a contingency table
pd.crosstab(Calls["marital"], Calls["education"])

education,primary,secondary,tertiary,unknown
marital,,,,
divorced,752,2815,1471,169
married,5246,13770,7038,1160
single,853,6617,4792,528


In [9]:
# More informative listing for a categorical variable
round(Calls.pivot_table(index=["job"]), 1)

,age,balance,campaign,day,duration,pdays,previous
job,,,,,,,
admin.,39.3,1135.8,2.6,15.6,246.9,47.9,0.7
blue-collar,40.0,1078.8,2.8,15.4,262.9,44.0,0.5
entrepreneur,42.2,1521.5,2.8,15.7,256.3,32.5,0.5
housemaid,46.4,1392.4,2.8,16.0,245.8,21.5,0.4
management,40.4,1763.6,2.9,16.1,254.0,38.7,0.7
retired,61.6,1984.2,2.3,15.4,287.4,37.4,0.6
self-employed,40.5,1648.0,2.9,16.0,268.2,34.7,0.6
services,38.7,997.1,2.7,15.6,259.3,42.0,0.5
student,26.5,1388.1,2.3,14.9,246.7,57.0,1.0


In [10]:
# Describe the numerical variables
round(Calls.describe(), 2)

,age,balance,day,duration,campaign,pdays,previous
count,45211.00,45211.00,45211.00,45211.00,45211.00,45211.00,45211.00
mean,40.94,1362.27,15.81,258.16,2.76,40.20,0.58
std,10.62,3044.77,8.32,257.53,3.10,100.13,2.30
min,18.00,-8019.00,1.00,0.00,1.00,-1.00,0.00
25%,33.00,72.00,8.00,103.00,1.00,-1.00,0.00
50%,39.00,448.00,16.00,180.00,2.00,-1.00,0.00
75%,48.00,1428.00,21.00,319.00,3.00,-1.00,0.00
max,95.00,102127.00,31.00,4918.00,63.00,871.00,275.00
